# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [2]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [8]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, 
                 max_depth=None, sufficient_share=1.0, 
                 criterion='gini', max_features=None, 
                 min_samples_leaf=2.0):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.min_samples_leaf = min_samples_leaf
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - np.sum(l_c ** 2 / l_s + r_c ** 2 / r_s, axis=1, keepdims=True) / (l_s + r_s)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return -(np.sum(l_c * np.log(l_c / l_s) + r_c * np.log(r_c / r_s), axis=1, keepdims=True)) / (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')        
        return 1 - (np.max(l_c, axis=1, keepdims=True) + np.max(r_c, axis=1, keepdims=True)) / (l_s + r_s)
            

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.sqrt(n_feature).astype('int')]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.log2(n_feature).astype('int')]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        ### В {sorted_x} кладётся отсортированный {x}, 
        ### а в {sorted_y} кладутся соответствующие этим отсортированным {x} значения {y}
        ### {class_number} --- кол-во различных классов (кол-во различных {y})
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        ### Скорее всего здесь вместо {self.min_samples_split} 
        ### имелось в виду {self.min_samples_leaf}
        splitted_sorted_y = sorted_y[self.min_samples_leaf : -self.min_samples_leaf]
        ### {r_border_ids} находит правые границы (+1), 
        ### повторяющихся последовательностей в {splitted_sorted_y},
        ### т.е. граница будет указывать на следующий после крайнего эл-та в повторяющиейся послед-ти,        
        ### а затем проецирует их позиции на позиции в {sorted_y}
        ### Вообще, {r_border_ids} указывает на те позиции, в которых мы можем производить разбиения
        ### т.е. возможные позиции threshold
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] \
                        + (self.min_samples_leaf + 1)
        ### если в {splitted_sorted_y} содержатся только одинаковые классы, то {r_border_ids} = []
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        ### В {eq_el_count} хранится кол-во элементов в каждой группе 
        ### повторяющихся послед-тей в {splitted_sorted_y},
        ### т.к. каждая границы {r_border_ids[i]} является окончанием одной 
        ### повторяющейся послед-ти в {splitted_sorted_y} и началом следующей
        ### то индексы {r_border_ids[i]} мы можем связать с конкретным классом
        ### закодируем наши границы методом one-hot-encoding
        ### sorted_y[r_border_ids - 1] --- узнаем классы, к которым относится повторяющиеся послед-ти
        ### в {splitted_sorted_y} на [r_border_ids - 1] местах
        ### r_border_ids - 1 делаем потому, что r_border_ids указывает на следующий после крайнего эл-та
        ### в повторяющейся послед-ти
        ### в {class_increments} = {one_hot_code} поэлементно умноженный на кол-во элеменов в каждом классе
        ### и {class_increments} показывает не только то, к какому классу принадлежит данная граница,
        ### но ещё и кол-во эл-ов между границами в {r_border_ids} [r_border_ids[i - 1] : r_border_ids[i]]
        eq_el_count = r_border_ids - np.append([self.min_samples_leaf], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        ### Обновляем {class_increments} в соответствии с теми частями, что мы отрезали у {sorted_y},
        ### когда получали {splitted_sorted_y}
        ### скорее всего вместо 'np.bincount(y[:self.min_samples_split], minlength=class_number)'
        ### надо использовать 'np.bincount(sorted_y[:self.min_samples_leaf], minlength=class_number)'
        class_increments[0] = class_increments[0] \
                            + np.bincount(sorted_y[:self.min_samples_leaf], minlength=class_number)
        
        # Что делает этот блок кода?
        ### {l_class_count} --- кол-во эл-ов каждого класса, которые попадут в левую часть разбения
        ### {r_class_count} --- кол-во эл-ов каждого класса, которые попадут в правую часть разбения
        ### {l_sizes} --- кол-во эл-ов, которые попадут в левую часть разбения
        ### {r_sizes} --- кол-во эл-ов, которые попадут в левую часть разбения
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        ### Вычисляем impurity measure для каждого вохможного разбиения
        ### и находим такое разбиение, где impurity measure -> min
        arr_G = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(arr_G)

        
        # Что делает этот блок кода?
        ### находим id граничного элемента
        left_el_id = l_sizes[idx][0]
        ### возвращаем значение impurity measure и threshold
        return arr_G[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        if x.shape[0] < self.min_samples_split \
            or depth >= self.max_depth \
            or np.max(np.bincount(y).astype('float')) / y.shape[0] > self.sufficient_share:
            return self.__make_leaf_node(y, node_id)
        
        
        gaints = np.empty(x.shape[1])
        thresholds = np.empty(x.shape[1])
        
        for f in self.get_feature_ids(x.shape[1]):
            g, threshold = self.__find_threshold(x[:, f], y)
            gaints[f] = g
            thresholds[f] = threshold
        g_minPos = np.argmin(gaints)
        
        
        if np.isnan(thresholds[g_minPos]):
            return self.__make_leaf_node(y, node_id)
        
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, g_minPos, thresholds[g_minPos])
        

        if y_l.shape[0] == 0 or y_r.shape[0] == 0:
            return self.__make_leaf_node(y, node_id)
        
        self.tree[node_id] = (self.NON_LEAF_TYPE, g_minPos, thresholds[g_minPos])
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)            
        
        
    def __make_leaf_node(self, y, node_id):
        cntElemEachClass = np.bincount(y, minlength=self.num_class)
        p = cntElemEachClass / (1.0 * cntElemEachClass.sum())
        self.tree[node_id] = (self.LEAF_TYPE, np.argmax(cntElemEachClass), p)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        min_y = np.min(y)
        y -= min_y
        self.__fit_node(x, y, 0, 0) 
        y += min_y

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [9]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [10]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [11]:
my_clf = MyDecisionTreeClassifier(min_samples_split=3, min_samples_leaf=2, max_depth=5, criterion='gini')
clf = DecisionTreeClassifier(min_samples_split=3, min_samples_leaf=2, max_depth=5, criterion='gini')

## Проверка скорости работы

In [12]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.635117053986
0.695672035217


## Проверка качества работы

In [13]:
gkf = KFold(n_splits=5, shuffle=True)

In [14]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.936143676727
0.931861644633
0.933150411574
0.931071755217
0.933771255145


In [15]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.933316704082
0.93277625343
0.932277375904
0.93248524154
0.935434249366
